# Texte et distance de jaccard

On va dans ce TD regarder les proximités de fables de la Fontaine en comparant les mots les composants.

Ceci est une prémière approche dans l'analyse de textes. Des méthodes bien plus *sioux* sont utilisées en vrai (mettre les mots au singulier, les verbe à l'infinitif, etc)

## création de la distance

Pour créer une distance entre fable on devra :

1. récupérer les fables
2. extraire les mots qui les composent
3. créer une distance

### Récupération des fables

Les fables sont disponibles là : https://www.gutenberg.org/files/56327/56327-h/56327-h.htm

On va récupérer le fichier html puis entraire le texte d celui-ci

#### page internet

En utilisant la bibliothèque [requests](https://requests-fr.readthedocs.io/en/latest/)

In [ ]:
import requests

In [ ]:
page = requests.get("https://www.gutenberg.org/files/56327/56327-h/56327-h.htm")

#### parser la page

Il faut récuperer les tokens html de la page qui compoent les fables.

En regardant la page dans un browser et les outils de développement on voit que :
* les titres des fables sont des des balises `<h3>`
* le texte des fables est dans le `<div>` juste en dessous (un `div` *frère*)

Il existe plusieurs bibliothèque pour parser du html. On va utiliser [beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

**Références** :
* https://www.dataquest.io/blog/web-scraping-tutorial-python/
* http://sametmax.com/parser-du-html-avec-beautifulsoup/

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
[type(item) for item in list(soup.children)]

On cherche tous les fables. En regardant dans le html on remarque que les `<h3>` semblent correspondre aux titres des fables. On vérifie :

In [ ]:
for x in soup.find_all('h3'):
    print(x)

Le contenu est dans le `div` juste après (c'est un frère) :

In [ ]:
for x in soup.find_all('h3'):
    print(x.text)
    print('----')
    poem = x.find_next_sibling()
    print(poem)
    print("=====")

On doit maintenant récupérer le poème. Il est dans une balise `span` de class différente à `pagenum`.

**Attention** : le dernier poème n'a pas de contenu. On doit traiter ce cas particulier pour qu'il n'y ait pas d'erreurs.

In [ ]:
for x in soup.find_all('h3'):
    print(x.text)
    print('----')
    poem = x.find_next_sibling()
    contenu_div = poem.find('div')
    if contenu_div is None: # le dernier poeme n'a pas de contenu
        continue
    for y in contenu_div.find_all('span'):
        if 'pagenum' not in y.attrs['class']:
            print(y.text)
    print("====")

On peut maintenant créer nos fables :

In [ ]:
fables = {}

for x in soup.find_all('h3'):
    titre = x.text
    div_frere = x.find_next_sibling()
    contenu_div = div_frere.find('div')    
    if contenu_div is None: # le dernier poeme n'a pas de contenu
        continue
        
    contenu = ""
    for y in contenu_div.find_all('span'):
        if 'pagenum' not in y.attrs['class']:
            contenu += y.text + '\n'
    fables[titre] = contenu
    
fables

## extraire les mots des fables

Pour cela on va comencer par mettre le texte en minuscule puis utiliser une expression régulière pour chercher tous les mots.

**Références** :
* https://fr.wikipedia.org/wiki/Expression_régulière
* https://www.lucaswillems.com/fr/articles/25/tutoriel-pour-maitriser-les-expressions-regulieres



In [ ]:
import re

In [ ]:
bag_of_word = {}
for titre, texte in fables.items():
    bag_of_word[titre] = set(re.findall(r'\b\w+\b',texte.lower()))

In [ ]:
print(fables['LE RENARD ET LES POULETS D’INDE.'])

In [ ]:
bag_of_word['LE RENARD ET LES POULETS D’INDE.']

On supprime les mots de longueur pus petit ou égal à 2 pour essayer de ne conserver que les *vrais* mots.

In [ ]:
for titre in bag_of_word:
    new = set()
    for x in bag_of_word[titre]:
        if len(x) > 2:
            new.add(x)
    bag_of_word[titre] = new

### distance de jaccard

La  [distance de jaccard](https://fr.wikipedia.org/wiki/Indice_et_distance_de_Jaccard) est très utilisée pour comparer des objets décrit par des ensembles de propriétés. 

Elle vaut si :
* 1 : aucun mots en commun
* 0 : même vocabulaire

On commence par regarder les distance avec une fable de référence :

In [ ]:
base = bag_of_word['LE RENARD ET LES POULETS D’INDE.']

similarite = []
for titre, texte in bag_of_word.items(): 
    similarite.append([1 - len(base.intersection(texte)) / len(base.union(texte)), titre])
    
similarite.sort()

In [ ]:
similarite

On crée la matrice de distance :

In [ ]:
elements = list(bag_of_word.keys())

d = []

for x in elements:
    d.append([])
    base = bag_of_word[x]
    for y in elements:
        texte = bag_of_word[y]
        d[-1].append(1 - len(base.intersection(texte)) / len(base.union(texte)))
d

## Analyse

**A vous** : 
* faites un MDS en 2 dimension de ces données
* tentez une isomap en changeant le nombre de voisins à prendre en compte
* faites une hiérarchie.

### MDS

In [ ]:
from sklearn import manifold

mds = manifold.MDS(n_components=2, max_iter=3000,
                   dissimilarity="precomputed", n_jobs=1)

pos = mds.fit(d).embedding_

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

In [ ]:
data = pd.DataFrame(pos)
data.columns=(str(x) for x in data.columns)

fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x='0', 
                y='1', 
                data=data,
                legend=False,
                ax=ax)
plt.show()

### isomap

In [ ]:
mds = manifold.Isomap(n_neighbors=2,
                      n_components=2, max_iter=3000,                      
                      metric="precomputed")

pos = mds.fit(d).embedding_

In [ ]:
data = pd.DataFrame(pos)
data.columns=(str(x) for x in data.columns)


fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x='0', 
                y='1', 
                data=data,
                legend=False,
                ax=ax)
plt.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=4,
                n_init=10, 
                max_iter=300).fit(data)

centers = kmeans.cluster_centers_
clusters_kmeans = kmeans.predict(data)

In [ ]:
current_palette = sns.color_palette()
colors = {i: current_palette[i] for i in range(len(centers))}

fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x='0', 
                y='1', 
                data=data,
                hue=clusters_kmeans,
                palette=colors,
                ax=ax)

plt.show()

In [ ]:
# petite classe en bas à droite.

numero_classe = 2 # vérifier que c'est bien le bon numéro, il change à chaque lanement

index = []
for i in range(len(elements)):
    if (clusters_kmeans[i] == numero_classe):
        index.append(i)
        print(elements[i])

In [ ]:
for p in range(len(index)):
    i = index[p]
    for j in index[p+1:]:
        print(elements[i], '-', elements[j])
        print("distance :", d[i][j])
        print("commun :", bag_of_word[elements[i]].intersection(bag_of_word[elements[j]]))

### hierarchie

In [ ]:
import sklearn
from sklearn.cluster import AgglomerativeClustering

In [ ]:
# pas euclidien, donc average linkage et affinity=precomputed

clustering = AgglomerativeClustering(n_clusters=None, 
                                     distance_threshold=0, linkage="average",
                                     affinity="precomputed").fit(d)

In [ ]:
fils = clustering.children_
hauteur = clustering.distances_
n = len(clustering.labels_)


node_position = pd.DataFrame([(0, 0) for i in range(n)], columns=['x', 'y'])

node_position 

pos = [0]
def backtracking(noeud):
    for x in noeud:
        if x < n:
            node_position.loc[x , 'x'] = pos[0]
            pos[0] += 1
        else:
            backtracking(fils[x - n])

backtracking(fils[-1])

for i, (son1, son2) in enumerate(fils):
    pos = 0.5 * (node_position.loc[son1]['x'] + node_position.loc[son2]['x'])
    node_position = node_position.append(pd.DataFrame([(pos, hauteur[i])], columns=['x', 'y']), ignore_index=True)

In [ ]:
import matplotlib.lines as mlines

In [ ]:
n = len(clustering.labels_)

fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x='x', 
                y='y', 
                data=node_position,
                legend=False,
                ax=ax)

for i, sons in enumerate(fils):
    u = node_position.loc[i + n]    
    v1 = node_position.loc[sons[0]]
    v2 = node_position.loc[sons[1]]
    
    l = mlines.Line2D([u['x'], v1['x']] , [u['y'], v1['y']])
    ax.add_line(l)
    l = mlines.Line2D([u['x'], v2['x']] , [u['y'], v2['y']])
    ax.add_line(l)


for i, row in node_position.iterrows():
    ax.text(row['x'], row['y'], i)
    
plt.show()

On ne peut pas en tirer grand chose. On regarde alors la hauteur d'agrégation pour donner une coupe

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

sns.lineplot(x=list(range(len(clustering.distances_))), 
                y=clustering.distances_, 
                legend=False,
                ax=ax)

plt.show()

On peut couper à 0.875 par exemple

### partition d'une hiérarchie

In [ ]:
partition = AgglomerativeClustering(n_clusters=None,
                                    compute_full_tree=True,
                                    distance_threshold=0.875,
                                    linkage="average",
                                    affinity="precomputed").fit(d)

In [ ]:
partition.labels_

In [ ]:
from collections import Counter

for cluster, number in Counter(partition.labels_).items():
    if number > 1:
        print(cluster, ":", number,"-", partition.distances_[cluster])

Une toute petite classe de 8 éléments :

In [ ]:
classe_taille_8 = 1 # vérifier que c'est bien le bon numéro, il change à chaque lanement
index = []
for i in range(len(elements)):
    if (partition.labels_[i] == classe_taille_8):
        index.append(i)
        print(elements[i])

In [ ]:
for p in range(len(index)):
    i = index[p]
    for j in index[p+1:]:
        print(elements[i], '-', elements[j])
        print("distance :", d[i][j])
        print("commun :", bag_of_word[elements[i]].intersection(bag_of_word[elements[j]]))